# Setup

In [ ]:
# All import statements needed for the project
import json
import requests
import pandas as pd
import urllib.parse
import requests
import pathlib
from pathlib import Path
import psycopg2
import geoalchemy2 as gdb
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import shapely
import sqlalchemy as db
import matplotlib as mpl
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib.animation as animation
import unittest

from sqlalchemy.orm import declarative_base

In [ ]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
#DATA_DIR = pathlib.Path("data")
#ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "ZIP_CODE_040114.shp"
#ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"
ZIPCODE_DATA_FILE = 'data/zipcodes/nyc_zipcodes.shp'
RENT_DATA_FILE = 'data/zillow_rent_data.csv'

NYC_DATA_APP_TOKEN = "qusZQnQU0ua9VbjGpXFDVhsbK"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "FINAL_PROJECT_DATA"
DB_USER = "leticialx"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [ ]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

# Part 1: Data Preprocessing

In [ ]:
def download_nyc_geojson_data(url: str,
                              file_name: str, 
                              force: bool = False) -> gpd.GeoDataFrame:
    """
    Download NYC GeoJSON data from a provided URL and saves it to a file.
    
    Keyword arguments:
    url (str) -- the URL from which the GeoJSON data will be downloaded
    file_name (str) -- the name of the file (without extension) to save the GeoJSON data
    force (bool, optional) -- if True, forces re-download of the file even if it exists locally (default is false)
    """
    
    filename = Path(file_name + '.geojson')
    
    if force or not filename.exists():
        print(f"Downloading {url} to {file_name}...")
    
        response = requests.get(url)
        text = response.text

        with open(file_name + '.geojson', 'w') as file:
            file.write(text)
        print(f"Done downloading {url}.")
        
    else:
        print(f"Reading from {file_name}...")
    
    gdf = gpd.read_file(file_name + '.geojson', driver = 'GeoJSON')
    return gdf

In [ ]:
def download_and_clean_311_data() -> gpd.GeoDataFrame:
    """Download and clean 311 complaints data."""
    
    url = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$$app_token=RbFfvU4T8a7C7rDHaA9eqAkvZ"
    
    file_name = 'complaints_head(10000)'
    complaints = download_nyc_geojson_data(url, file_name, force=False)
    
    # choose the columns we neeed
    new_complaints = complaints[['created_date', 'incident_zip', 'complaint_type', 'geometry']]
    
    # normalize SRID to EPSG:4326
    new_complaints = new_complaints.to_crs(epsg=4326)
    
    # delete NaN
    columns_with_nan = new_complaints.columns[new_complaints.isnull().any()].tolist()
    
    for column_name in columns_with_nan:
        print("Processing column:", column_name)
        new_complaints = new_complaints.dropna(subset=[column_name])
        print("Removed rows with NaN value in", column_name)

    return new_complaints

In [ ]:
def download_and_clean_tree_data() -> gpd.GeoDataFrame:
    """Download and clean 2015 tree census data."""
    
    url = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson?$$app_token=RbFfvU4T8a7C7rDHaA9eqAkvZ"
    file_name = 'trees_head(10000)'
    trees = download_nyc_geojson_data(url, file_name, force=False)
    
    # normalize SRID to EPSG:4326
    trees = trees.to_crs(epsg=4326)
    
    # delete NaN
    columns_with_nan = trees.columns[trees.isnull().any()].tolist()
    
    for column_name in columns_with_nan:
        print("Processing column:", column_name)
        trees = trees.dropna(subset=[column_name])
        print("Removed rows with NaN value in", column_name)
        
    # choose the columns we neeed
    new_trees = trees[['tree_id', 'zipcode', 'spc_common', 'health', 'status','geometry']].copy()

    return new_trees

In [ ]:
def load_and_clean_zipcodes(zipcode_datafile: str) -> gpd.GeoDataFrame:
    """Load and clean NYC’s zip codes with geometric boundary data data."""
    
    zipcode_data = gpd.read_file(zipcode_datafile)
    
    zipcodes = zipcode_data[['ZIPCODE', 'geometry']]
    zipcodes = zipcodes.rename(columns={'ZIPCODE': 'zipcode'})
    
    # normalize SRID to EPSG:4326
    zipcodes = zipcodes.to_crs(epsg=4326)
    
    return zipcodes

In [ ]:
def load_and_clean_zillow_data(rent_datafile: str) -> pd.DataFrame:
    """Load and clean historical monthly average rents by zip code from Zillow."""
    
    zillow_rent_data = pd.read_csv(rent_datafile)
    
    columns_to_drop = ['RegionID', 'SizeRank', 'RegionType', 'State', 'StateName', 'City', 'Metro', 'CountyName']
    df_zillow_data = zillow_rent_data.drop(columns=columns_to_drop)
    
    # Normalize rent data
    df_zillow_data = df_zillow_data.rename(columns={'RegionName': 'rent_zip'})
    
    rents = pd.melt(df_zillow_data, id_vars=['rent_zip'], var_name='rent_date', value_name='rent_amount')
    
    rents['rent_date'] = pd.to_datetime(rents['rent_date'])
    
    # Drop NaNs
    rents = rents.dropna()
    
    return rents

In [ ]:
def load_all_data() -> (gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame, pd.DataFrame):
    """Load all data."""
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data(RENT_DATA_FILE)
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data = download_and_clean_311_data()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
# 记得删掉！！
tree = gpd.read_file('trees.geojson')
trees = tree.to_crs(epsg=4326)

# delete NaN
columns_with_nan = trees.columns[trees.isnull().any()].tolist()

for column_name in columns_with_nan:
    print("Processing column:", column_name)
    trees = trees.dropna(subset=[column_name])
    print("Removed rows with NaN value in", column_name)

# choose the columns we neeed
geodf_tree_data = trees[['tree_id', 'zipcode', 'spc_common', 'health', 'status','geometry']].copy()


In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

# Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

### Creating Tables

In [ ]:
engine = db.create_engine(DB_URL)

In [ ]:
# define the SQL statements to create 4 tables using SQL
ZIPCODE_SCHEMA = """
CREATE TABLE nyc_zip_codes (
    id SERIAL PRIMARY KEY,
    zip_code INTEGER,
    geom POLYGON
)
"""

NYC_311_SCHEMA = """
CREATE TABLE complaints_311 (
    id SERIAL PRIMARY KEY,
    created_date TIMESTAMP,
    incident_zip INTEGER,
    complaints_type TEXT,
    geometry POINT
)
"""

NYC_TREE_SCHEMA = """
CREATE TABLE trees (
    id SERIAL PRIMARY KEY,
    tree_id INTEGER,
    tree_zip INTEGER,
    spc_common TEXT,
    health TEXT,
    status TEXT,
    geometry POINT
)
"""

ZILLOW_SCHEMA = """
CREATE TABLE rents (
    id SERIAL PRIMARY KEY,
    rent_zip INTEGER,
    rent_date DATE,
    rent NUMERIC(6)
)
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# execute the schema files to create tables using SQL
with engine.connect() as connection:
    
    connection.execute("DROP TABLE IF EXISTS nyc_zip_codes, complaints_311, trees, rents CASCADE") # if table already exists
    
    connection.execute(ZIPCODE_SCHEMA)
    connection.execute(NYC_311_SCHEMA)
    connection.execute(NYC_TREE_SCHEMA)
    connection.execute(ZILLOW_SCHEMA)

### Add Data to Database

In [ ]:
# dataframe to sql table
df_zillow_data.to_sql('rents', con=engine, if_exists='replace', index=False)

# GeoDataFrame data into a PostGIS-enabled PostgreSQL table
geodf_311_data.to_postgis('complaints', con=engine, if_exists='replace', index=False)
geodf_zipcode_data.to_postgis('zipcodes', con=engine, if_exists='replace', index=False)
geodf_tree_data.to_postgis('trees', con=engine, if_exists='replace', index=False)

# Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query: str, outfile: str):
    with open(outfile, "w") as f:
        f.write(query)

## Query 1: Which area might be more calm to live in?
Between October 1st, 2022 and September 30th, 2023 (inclusive), find the number of 311 complaints per zip code. 

The query result should have two columns, one row per zip code, with the number of complaints in descending order.


In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "num_complaints_by_zipcode"

QUERY_1 = """
SELECT incident_zip, COUNT(*) AS complaints_count
FROM complaints
WHERE
    created_date BETWEEN '2022-10-01T00:00:00.000' AND '2023-09-30T23:59:59.999'
GROUP BY incident_zip
"""

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

In [ ]:
# execute the query
with engine.connect() as connection:
    result = connection.execute(QUERY_1)

for row in result:
    print(row)

## Query 2: Where has the most greenery?
Using just the trees table, which 10 zip codes have the most trees?

The query result should have two columns, 10 rows. The rows should be sorted by the total number of trees, descending.


In [ ]:
QUERY_2_FILENAME = QUERY_DIR / "top_10_zipcodes_with_most_trees"

# table column name is the same as dataframe, not schema!!!
QUERY_2 = """
SELECT zipcode, COUNT(*) AS total_trees
FROM trees
GROUP BY zipcode
ORDER BY total_trees DESC
LIMIT 10;
"""

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

In [ ]:
# execute the query
with engine.connect() as connection:
    result = connection.execute(QUERY_2)

for row in result:
    print(row)

## Query 3: Can I afford a place in the areas with the most trees?
Of the 10 zip codes with the most trees, for the month of August 2023, what is the average rent by zip code?

The query should have a JOIN statement. The query result should have two columns (not three) and 10 rows. The rows should be sorted by the total number of trees, descending. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128. 

In [ ]:
QUERY_3_FILENAME = QUERY_DIR / "avg_rent_for_top_10_zipcodes_with_most_trees"

QUERY_3 = """
WITH top10_zipcodes AS (
    SELECT zipcode, COUNT(*) AS total_trees
    FROM trees
    GROUP BY zipcode
    ORDER BY total_trees DESC
    LIMIT 10
)

SELECT t.zipcode, ROUND(AVG(r.rent_amount)::numeric, 2) AS average_rent
FROM rents r
JOIN trees t ON t.zipcode = r.rent_zip::text
WHERE DATE_PART('month', r.rent_date) = 8
    AND DATE_PART('year', r.rent_date) = 2023
    AND r.rent_zip::text IN (SELECT zipcode FROM top10_zipcodes)
GROUP BY t.zipcode
ORDER BY (SELECT total_trees FROM top10_zipcodes WHERE zipcode = t.zipcode) DESC;
"""

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

In [ ]:
# execute the query
with engine.connect() as connection:
    result = connection.execute(QUERY_3)

for row in result:
    print(row)

## Query 4: Could there be a correlation between an area’s rent, the number of its trees, and the number of 311 complaints?
For the month of January 2023, return the 5 zip codes with the lowest average rent, and 5 zipcodes of the highest average rent, and include the tree count and complaint count for each zip code by using JOIN statements.

The query result should have 4 columns (zip code, average rent, tree count, and complaint count) and 10 rows: five with the highest average rent, and five with the lowest average rent. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128.

In [ ]:
QUERY_4_FILENAME = QUERY_DIR / "rent_trees_complaints_correlation"

QUERY_4 = """
WITH RentData AS (
    SELECT CAST(r.rent_zip AS TEXT) AS zipcode, 
           CAST(AVG(r.rent_amount) AS DECIMAL(10, 2)) AS average_rent
    FROM rents r
    WHERE DATE_PART('month', r.rent_date) = 1
      AND DATE_PART('year', r.rent_date) = 2023
    GROUP BY r.rent_zip
),
TreeData AS (
    SELECT zipcode, COUNT(*) AS tree_count
    FROM trees
    GROUP BY zipcode
),
ComplaintData AS (
    SELECT incident_zip AS zipcode, COUNT(*) AS complaint_count
    FROM complaints
    WHERE DATE_PART('month', created_date) = 1
      AND DATE_PART('year', created_date) = 2023
    GROUP BY incident_zip
),
LowestRent AS (
    SELECT * FROM RentData
    ORDER BY average_rent ASC
    LIMIT 5
),
HighestRent AS (
    SELECT * FROM RentData
    ORDER BY average_rent DESC
    LIMIT 5
)
SELECT lr.zipcode,
       lr.average_rent AS formatted_rent,
       COALESCE(td.tree_count, 0) AS tree_count,
       COALESCE(cd.complaint_count, 0) AS complaint_count
FROM LowestRent lr
LEFT JOIN TreeData td ON lr.zipcode = td.zipcode
LEFT JOIN ComplaintData cd ON lr.zipcode = cd.zipcode
UNION ALL
SELECT hr.zipcode,
       hr.average_rent AS formatted_rent,
       COALESCE(td.tree_count, 0) AS tree_count,
       COALESCE(cd.complaint_count, 0) AS complaint_count
FROM HighestRent hr
LEFT JOIN TreeData td ON hr.zipcode = td.zipcode
LEFT JOIN ComplaintData cd ON hr.zipcode = cd.zipcode;
"""

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

In [ ]:
# execute the query
with engine.connect() as connection:
    result = connection.execute(QUERY_4)

for row in result:
    print(row)

## Query 5: Where has the most greenery (take 2)?
Rewrite Query 2 to use both the trees table and the zipcodes table. Join both tables where the coordinate point of the tree is inside the polygon boundary of the zipcode as defined in the zipcode table.

The query should have a JOIN statement. The query results should match exactly the results of Query 2.

In [ ]:
QUERY_5_FILENAME = QUERY_DIR / "top_10_zipcodes_with_most_trees_by_geometry"

QUERY_5 = """
SELECT z.zipcode, COUNT(*) AS total_trees
FROM zipcodes z
JOIN trees AS t ON ST_Contains(z.geometry, t.geometry)
GROUP BY z.zipcode
ORDER BY total_trees DESC
LIMIT 10;
"""

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

In [ ]:
# execute the query
with engine.connect() as connection:
    result = connection.execute(QUERY_5)

for row in result:
    print(row)

## Query 6: What is the immediate area like?
Using the following coordinate pair on campus, which trees are within ½ mile radius of this point?
Latitude: 40.80737875669467, Longitude: -73.96253174434912

The result should have 5 columns (ID, species, health, status, and coordinate location of each tree).

You may use regular Python to help construct the geometric object needed for setting the radius in the query.


In [ ]:
target_latitude = 40.80737875669467
target_longitude = -73.96253174434912

target_point = Point(target_longitude, target_latitude)

# Create a circle with a radius of 0.5 miles around the target location
radius = 0.5  # in miles
circle_buffer = target_point.buffer(radius / 69)  # Assuming 1 degree of latitude is approximately 69 miles

gdf_circle = gpd.GeoDataFrame(geometry=[circle_buffer])

In [ ]:
QUERY_6_FILENAME = QUERY_DIR / "trees_within_coordinate_pair"

QUERY_6 = f"""
SELECT tree_id, spc_common, health, status, ST_AsText(geometry) AS coordinate_location
FROM trees
WHERE ST_Intersects(geometry, ST_GeomFromText('{circle_buffer.wkt}', 4326))
"""

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

In [ ]:
# execute the query
with engine.connect() as connection:
    result = connection.execute(QUERY_5)

for row in result:
    print(row)

# Part 4: Visualizing Data

In [ ]:
# Render graphics directly in the notebook without having to call plt.show
%matplotlib inline
plt.style.use("seaborn-dark")  # Choose a color palett

In [ ]:
# Query to select data
query1 = 'SELECT * FROM rents'
query2 = 'SELECT * FROM complaints'
query3 = 'SELECT * FROM trees'
query4 = 'SELECT * FROM zipcodes'

In [ ]:
# Read all data into pandas DataFrame and geopandas GeoDataFrame
rent_data = pd.read_sql(query1, engine)

complaint_data = gpd.read_postgis(query2, con=engine, geom_col='geometry')
tree_data = gpd.read_postgis(query3, con=engine, geom_col='geometry')
zipcode_data = gpd.read_postgis(query4, con=engine, geom_col='geometry')

## Visualization 1: What can I expect to put up with in NYC?
First, find the top 3 complaint types for October 1st, 2022 to September 30th, 2023 (inclusive). 

Then, create an appropriate visualization for the number of complaints per day over $timeframe for those complaint types.  

In [ ]:
# final data selection - skipped for now
def get_data_for_visual_1() -> pd.DataFrame:
    """Get the dataframe needed for visualization 1."""
    
    # Set time range
    start_date = pd.Timestamp('2022-10-01')
    end_date = pd.Timestamp('2023-09-30')
    # Select data within the time range
    complaint_data_filtered = complaint_data.loc[(complaint_data['created_date'] >= start_date) 
                              & (complaint_data['created_date'] <= end_date)]
    # Select data of top 3 complaint types
    top_3_types = complaint_data_filtered['complaint_type'].value_counts().nlargest(3).index.tolist()
    df_top3 =  complaint_data[complaint_data['complaint_type'].isin(top_3_types)].copy()
    # Group data by creation date
    df_top3['created_date'] = pd.to_datetime(df_top3['created_date'])
    df_top3_grouped = df_top3.groupby(df_top3['created_date'].dt.date)['incident_zip'].size().reset_index(name='count')
    
    return df_top3_grouped

In [ ]:
# test: use current data
top_3_types = complaint_data['complaint_type'].value_counts().nlargest(3).index.tolist()
df_top3 =  complaint_data[complaint_data['complaint_type'].isin(top_3_types)].copy()

df_top3['created_date'] = pd.to_datetime(df_top3['created_date'])

df_top3_grouped = df_top3.groupby(df_top3['created_date'].dt.date)['incident_zip'].size().reset_index(name='count')

In [ ]:
# visulization
def top_3_complaint_types(df_top3_grouped: pd.DataFrame):
    """Create the plot 'Number of Complaints per Day for Top 3 Complaint Types'."""
    
    df_top3_grouped.plot(kind='line', figsize=(10, 6))
    plt.title('Number of Complaints per Day for Top 3 Complaint Types')
    plt.xlabel('Date')
    plt.ylabel('Number of Complaints')
    plt.legend(title='Complaint Type')
    plt.grid(True)

In [ ]:
top_3_complaint_types(df_top3_grouped)

### Extra credit 1: animation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Function to update the plot for each frame in the animation
def update(frame: int):
    """
    A Function for the animation
    
    Parameters:
    frame(int): Frame number for the animation
    """
    ax.clear()
    ax.plot(df_top3_grouped.index[:frame+1], df_top3_grouped.values[:frame+1])
    ax.set_title('Number of Complaints per Day')
    ax.set_xlabel('Date')
    ax.set_ylabel('Number of Complaints')
    ax.grid(True)

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(df_top3_grouped), interval=100, repeat=False)

## Visualization 2: What are the most common complaints in the immediate area?¶
Create a visualization that shows the number of complaints by complaint type for the top 10 complaints in zip code 10027 for October 1st, 2018 to September 30th, 2023 (inclusive).

In [ ]:
# final data selection - skipped for now
def get_data_for_visual_2() -> pd.Series:
    """Get the dataframe needed for visualization 2."""
    
    # Set time range
    start_date = pd.Timestamp('2018-10-01')
    end_date = pd.Timestamp('2023-09-30')
    # Select the data within the time range in zip code 10027 
    df_zipcode_10027 = complaint_data.loc[(complaint_data['incident_zip'] == 10027.0) &
                          (complaint_data['created_date'] >= start_date) &
                          (complaint_data['created_date'] <= end_date)]
    # Get the top 10 complaint types
    df_top10 = complaint_data['complaint_type'].value_counts().nlargest(10)
    
    return df_top10

In [ ]:
# test: use current data
df_top10 = complaint_data['complaint_type'].value_counts().nlargest(10)

In [ ]:
# visualization
def top_10_complaint_types_10027(df_top10: pd.Series):
    """Greate the histogram of 'Top 10 Complaint Types in Zip Code 10027'."""
    
    plt.figure(figsize=(10, 6))
    df_top10.plot(kind='bar', color='skyblue')
    
    plt.xlabel('Complaint Type')
    plt.ylabel('Frequency')
    plt.title('Top 10 Complaint Types in Zip Code 10027')
    
    plt.xticks(rotation=45)

In [ ]:
top_10_complaint_types_10027(df_top10)

## Visualization 3: Is there any correlation between rent, trees, and complaints at all?
Between January 1st, 2015 and September 30th, 2023 (inclusive), create a visualization using 2 subplots that share the x-axis where one subplot shows rent compared to the number of trees by zip code, and the other subplot shows rent compared to the number of complaints by zip code.

In [ ]:
# final data selection - skipped for now
def get_data_for_visual_3() -> pd.DataFrame:
    """Get the dataframe needed for visualization 3."""
    
    # Select the rent data within time range
    rent_data_filtered = rent_data[(rent_data['rent_date'] >= '2015-01-01') 
                                   & (rent_data['rent_date'] <= '2023-09-30')]
    # Match column data type
    rent_data_filtered['rent_zip'] = rent_data_filtered['rent_zip'].astype(str) 
    
    # Select the complaint data within time range
    complaint_data_filtered = complaint_data[(complaint_data['created_date'] >= '2015-01-01') 
                                             & (complaint_data['created_date'] <= '2023-09-30')]
    # Group complaints and trees by zipcode
    complaint_by_zip = complaint_data_filtered.groupby('incident_zip').size().reset_index(name='num_complaints')
    trees_by_zip = tree_data.groupby('zipcode').size().reset_index(name='num_trees')
    # Merge filtered data frame
    merged_data = rent_data_filtered.merge(trees_by_zip, left_on='rent_zip', right_on='zipcode', how='inner')
    merged_data = merged_data.merge(complaint_by_zip, left_on='rent_zip', right_on='incident_zip', how='inner')
    
    return merged_data

In [ ]:
# test:
# rent_data_filtered = rent_data[(rent_data['rent_date'] >= '2015-01-01') & (rent_data['rent_date'] <= '2023-09-30')]
rent_data['rent_zip'] = rent_data['rent_zip'].astype(str) # match column data type

# complaint_data_filtered = complaint_data[(complaint_data['created_date'] >= '2015-01-01') & (complaint_data['created_date'] <= '2023-09-30')]
complaint_by_zip = complaint_data.groupby('incident_zip').size().reset_index(name='num_complaints')

trees_by_zip = tree_data.groupby('zipcode').size().reset_index(name='num_trees')

merged_data = rent_data.merge(trees_by_zip, left_on='rent_zip', right_on='zipcode', how='inner')
merged_data = merged_data.merge(complaint_by_zip, left_on='rent_zip', right_on='incident_zip', how='inner')

In [ ]:
# visualization
def rent_trees_complaints_correlation(merged_data: pd.DataFrame):
    """Create the 2 subplots on the correlation between rent, trees, and complaints."""
    
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 8))
    
    ax1.scatter(merged_data['rent_amount'], merged_data['num_trees'])
    ax1.set_ylabel('Number of Trees')
    
    ax2.scatter(merged_data['rent_amount'], merged_data['num_complaints'])
    ax2.set_ylabel('Number of Complaints')
    
    plt.xlabel('Rent')

In [ ]:
rent_trees_complaints_correlation(merged_data)

## Visualization 4: If I can afford more in rent, will there be fewer issues & complaints?
Create a boxplot, where the x-axis is average rent in September 2023, separated into $1000 bins (i.e. $0-1000, $1001-2000, etc), and the y-axis is the number of 311 complaints observed in each zip code between October 1, 2022 (inclusive) to September 30, 2023 (inclusive).

In [ ]:
# final data selection - skipped for now
def get_data_for_visual_4() -> pd.DataFrame:
    """Get the dataframe needed for visualization 4."""
    
    # Define bin ranges and labels for rent amounts
    bins = [0, 1000, 2000, 3000, 4000, np.inf]
    bin_labels = ['0-1000', '1001-2000', '2001-3000', '3001-4000', '4001+']
    
    # Match column data type
    rent_data['rent_zip'] = rent_data['rent_zip'].astype(str)
    # Select data within the time range and divide into bins
    rent_data['rent_bin'] = pd.cut(rent_data['rent_amount'], bins=bins, labels=bin_labels)
    rent_data_sept_2023 = rent_data[(rent_data['rent_date'] <= '2023-09-30')
                                    & (rent_data['rent_date'] >= '2023-09-01')]
    average_rent_by_zipcode = rent_data_sept_2023.groupby('rent_zip')['rent_amount'].mean().reset_index()
    average_rent_by_zipcode.columns = ['rent_zip', 'average_rent']
    average_rent_by_zipcode['rent_bin'] = pd.cut(average_rent_by_zipcode['average_rent'], bins=bins, labels=bin_labels)
    
    # Filter and groupby complaint data
    complaints_data_filtered = complaint_data[(complaint_data['created_date'] >= '2022-10-01') 
                                              & (complaint_data['created_date'] <= '2023-09-30')]
    complaints_by_zip = complaints_data_filtered.groupby('incident_zip').size().reset_index(name='num_complaints')
    
    # Merge data
    merged_data = rent_data_sept_2023.merge(complaints_by_zip, left_on='rent_zip', right_on='incident_zip', how='inner')
    
    return merged_data

In [ ]:
# test:   
#rent_data_sept_2023 = rent_data[(rent_data['rent_date'] >= '2023-09-01') & (rent_data['rent_date'] <= '2023-09-30')]
bins = [0, 1000, 2000, 3000, 4000, np.inf]
bin_labels = ['0-1000', '1001-2000', '2001-3000', '3001-4000', '4001+']

rent_data['rent_zip'] = rent_data['rent_zip'].astype(str) # match column data type

# Select data within the time range and divide into bins
rent_data_sept_2023 = rent_data[(rent_data['rent_date'] <= '2023-09-30')
                                & (rent_data['rent_date'] >= '2023-09-01')].groupby('rent_zip')['rent_amount'].mean().reset_index()

average_rent_by_zipcode = rent_data_sept_2023.groupby('rent_zip')['rent_amount'].mean().reset_index()
average_rent_by_zipcode.columns = ['rent_zip', 'average_rent']
average_rent_by_zipcode['rent_bin'] = pd.cut(average_rent_by_zipcode['average_rent'], bins=bins, labels=bin_labels)

#complaints_data_filtered = complaints_data[(complaints_data['created_date'] >= '2022-10-01') & (complaints_data['created_date'] <= '2023-09-30')]

complaints_by_zip = complaint_data.groupby('incident_zip').size().reset_index(name='num_complaints')

merged_data = average_rent_by_zipcode.merge(complaints_by_zip, left_on='rent_zip', right_on='incident_zip', how='inner')

In [ ]:
# visulization
def rent_complaints_correlation(merged_data: pd.DataFrame):
    """Create the plot 'Boxplot of Rent vs. 311 Complaints'."""
    
    plt.figure(figsize=(10, 6))
    plt.boxplot([merged_data[merged_data['rent_bin'] == label]['num_complaints'] for label in bin_labels], labels=bin_labels)
    plt.xlabel('Average Rent in September 2023 (Bins)')
    plt.ylabel('Number of 311 Complaints (Oct 1, 2022 - Sep 30, 2023)')
    plt.title('Boxplot of Rent vs. 311 Complaints')

In [ ]:
rent_complaints_correlation(merged_data)

## Visualization 5: Where were the recent 311 incidents reported from in the immediate area?
Create a geospatial plot of the coordinates of reported 311 incidents that happened between January 1st, 2023 and September 30th, 2023 (inclusive) within a 1 kilometer radius of the same coordinate from Query 6 in Part 3.

In [ ]:
# final data selection - skipped for now
def get_data_for_visual_5() -> gpd.GeoDataFrame:
    """Get the dataframe needed for visualization 5."""
    # Select data within time range
    complaints_data_filtered = complaint_data[(complaint_data['created_date'] >= '2023-01-01') 
                                              & (complaint_data['created_date'] <= '2023-09-30')]
    
    # Create a Point geometry of the target location
    target_latitude = 40.80737875669467
    target_longitude = -73.96253174434912
    target_point = Point(target_longitude, target_latitude)
    
    # Calculate distance between incidents and the target location
    complaints_data_filtered['distance_to_target'] = complaints_data_filtered['geometry'].distance(target_point)
    
    # Filter incidents within a 1-kilometer radius of the target location
    df_incidents_within_radius = complaints_data_filtered[complaints_data_filtered['distance_to_target'] <= 1000]  # Distance in meters (1 kilometer = 1000 meters)
    
    return df_incidents_within_radius

In [ ]:
### test:

# Create a Point geometry
target_latitude = 40.80737875669467
target_longitude = -73.96253174434912

target_point = Point(target_longitude, target_latitude)

# Calculate distance between incidents and the target location
complaint_data['distance_to_target'] = complaint_data['geometry'].distance(target_point)

# Filter incidents within a 1-kilometer radius of the target location
df_incidents_within_radius = complaint_data[complaint_data['distance_to_target'] <= 10000]  # Distance in meters (1 kilometer = 1000 meters)

In [ ]:
# visualization
def incidents_within_radius(df_incidents_within_radius: gpd.GeoDataFrame):
    """Greate the plot '311 Incidents within 1km Radius'."""
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    
    world.plot(ax=ax, color='lightgrey')
    df_incidents_within_radius.plot(ax=ax, markersize=10, color='red', marker='o', label='Incidents within 1km radius')
    
    ax.legend()
    plt.title('311 Incidents within 1km Radius')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

In [ ]:
incidents_within_radius(df_incidents_within_radius)

## Visualization 6: Are areas trying to improve the amount of trees in the neighborhood?
Create a geospatial plot of two sets of data: the coordinates of trees in NYC, and the coordinates of "New Tree Request" 311 complaint types that were made from October 1st, 2018 to September 30th, 2023 (inclusive). 

In [ ]:
# final data selection - skipped for now
def get_data_for_visual_6() -> gpd.GeoDataFrame:
    """Get the dataframe needed for visualization 6."""
    
    # Select "New Tree Request" complaints within the time range
    new_tree_requests = complaint_data[
        (complaint_data['complaint_type'] == 'New Tree Request') 
        & (complaint_data['created_date'] >= '2018-10-01') 
        & (complaint_data['created_date'] <= '2023-09-30')
]
    return new_tree_requests

In [ ]:
type(tree_data)

In [ ]:
def trees_and_new_trees_request(tree_data: gpd.GeoDataFrame, new_tree_requests: gpd.GeoDataFrame):
    """Create the plot 'Trees and New Tree Requests'."""
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    tree_data.plot(ax=ax, color='green', alpha=0.7, label='Trees')
    new_tree_requests.plot(ax=ax, color='blue', alpha=0.7, label='New Tree Requests')
    
    plt.title('Trees and New Tree Requests')
    plt.legend()

In [ ]:
# test:
trees_and_new_trees_request(tree_data, complaint_data)

## Extra Credit-2 UnitTests

Define at least one unit test for every function and class method you’ve defined.

In [ ]:
class TestPartOne(unittest.TestCase):  
    
    def test_download_nyc_geojson_data(self) -> None:  
        url = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$$app_token=RbFfvU4T8a7C7rDHaA9eqAkvZ&$limit=1000"
        file_name = 'complaints_(1000)'
        download_nyc_geojson_data(url, file_name, force)
        expected_file = 'complaints.geojson'
        file_exists = os.path.isfile(expected_file)
        self.assertTrue(file_exists, f"Didn't successfully download '{expected_file}'.")       

    def test_download_and_clean_311_data(self) -> None:
        result = download_and_clean_311_data()
        self.assertTrue(isinstance(result, gpd.GeoDataFrame))
        
    def test_download_and_clean_tree_data(self) -> None:
        result = download_and_clean_tree_data()
        expected_columns = ['tree_id', 'zipcode', 'spc_common', 'health', 'status','geometry']]
        for column in expected_columns:
            self.assertIn(column, result.columns)
            
    def test_load_and_clean_zipcodes(self) -> None:
        result = load_and_clean_zipcodes('data/zipcodes/nyc_zipcodes.shp')
        self.assertTrue(isinstance(result, gpd.GeoDataFrame))
        
    def test_load_and_clean_zillow_data(self) -> None:
        result = load_and_clean_zillow_data('data/zillow_rent_data.csv')
        self.assertTrue(isinstance(result, pd.DataFrame))
        
    def test_load_all_data(self) -> None:
        a, b, c, d = load_all_data()
        self.assertTrue(isinstance(a, gpd.GeoDataFrame))
        self.assertTrue(isinstance(b, gpd.GeoDataFrame))
        self.assertTrue(isinstance(c, gpd.GeoDataFrame))
        self.assertTrue(isinstance(d, pd.DataFrame))

In [ ]:
class TestPartTwo(unittest.TestCase):  
    def test_write_query_to_file(self) -> None:
        
        test_file = 'try.sql'
        write_query_to_file(
            "CREATE TABLE test_table (id INT, column_one VARCHAR(255))",
            test_file)
        
        self.assertTrue(os.path.isfile(test_file), f"Did not create file '{test_file}'

In [ ]:
# get thorough examination and detailed debugging during test execution
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], verbosity=3, exit = False)